In [1]:
%load_ext autoreload
%autoreload 2

In [233]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))
from policy import ConvNet, ResNet
from concepts import DynamicConcepts, linear_regression
import absl.logging
import json
import string
import random
import numpy as np
import os
from env import gogame
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}
import tensorflow as tf

absl.logging.set_verbosity(absl.logging.ERROR)

# Set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

session_name = "falcon"
model_name = "net"
board_size = 5
board_name = f'{board_size}x{board_size}'
agents_to_sample = [0, 10, 20, 60, 100, 500, 1000]
resnet = False
random_subpar_rollout = True

model_type = "resnet" if resnet else "convnet"

# Mcts simulations
simulations = 50

full_model_path = f"../models/saved_sessions/{model_type}/board_size_{board_size}/{session_name}/"

mcts_model_path = f"../models/best_models/{model_type}/board_size_{board_size}/net_1000.keras"

init_state, concept_type_single = DynamicConcepts.opening_play(board_size)

print("Initial state:\n", gogame.str(init_state))

dynamic_concept = DynamicConcepts(init_state, simulations, board_size, concept_type_single, mcts_model_path, random_subpar_rollout, resnet)

CONCEPT_NAME = DynamicConcepts.opening_play.__name__

Initial state:
 	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: BLACK, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 0, White Area: 0

INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpv5lixi69/assets


INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpv5lixi69/assets
2024-02-13 09:51:20.297960: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpv5lixi69
2024-02-13 09:51:20.300556: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-02-13 09:51:20.300568: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpv5lixi69
2024-02-13 09:51:20.310537: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-02-13 09:51:20.354332: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpv5lixi69
2024-02-13 09:51:20.370264: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 72305 microseconds.


In [237]:
positive_cases = []
negative_cases = []

pos, neg = dynamic_concept.generate_cases()

positive_cases.extend(pos)
negative_cases.extend(neg)

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

In [238]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (7, 5, 5, 5)
Negative cases:  (10, 5, 5, 5)


In [236]:
# If larger than 50, do not visualize the tree do to the size (crashes the kernel)
if simulations <= 100:
    graph = dynamic_concept.view_tree()

    graph.render('./visualization/images/tree', view=True)

In [36]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [215]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  0


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print(gogame.str(positive_cases[i], nn_format=True))
    print()

In [ ]:
# Print all the negative cases
for i in range(len(negative_cases)):
    print(f"Negative case {i}:")
    print(negative_cases[i])
    print(gogame.str(negative_cases[i], nn_format=True))
    print()

In [230]:
BINARY = True

# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

POSITIONS_TO_CONSIDER = points.shape[0]

print("Positions to consider: ", POSITIONS_TO_CONSIDER)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=True,
    epochs=100,
    dynamic=True
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/"):
    # Test if epoch folder exists
    if os.path.exists(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{name}"):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{name}"):
            os.remove(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{name}/{file}")

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/dynamic", exist_ok=True)
os.makedirs(f"../concept_presences/dynamic/{model_type}", exist_ok=True)
os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}", exist_ok=True)
os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}", exist_ok=True)
os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}", exist_ok=True)
os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{name}", exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{name}/{random_suffix}.json", "w") as f:
    json.dump(score, f)

Positions to consider:  21
Epoch 1/100


2024-02-13 09:47:35.704121: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 09:47:35.737304: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 789ms/step - loss: 0.8582
Epoch 2/100
1/1 [==============================] - 0s 82ms/step - loss: 0.8579
Epoch 3/100
1/1 [==============================] - 0s 99ms/step - loss: 0.8575
Epoch 4/100
1/1 [==============================] - 0s 78ms/step - loss: 0.8572
Epoch 5/100
1/1 [==============================] - 0s 79ms/step - loss: 0.8568
Epoch 6/100
1/1 [==============================] - 0s 53ms/step - loss: 0.8565
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 0.8562
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 0.8558
Epoch 9/100
1/1 [==============================] - 0s 23ms/step - loss: 0.8555
Epoch 10/100
1/1 [==============================] - 0s 23ms/step - loss: 0.8551
Epoch 11/100
1/1 [==============================] - 0s 23ms/step - loss: 0.8548
Epoch 12/100
1/1 [==============================] - 0s 33ms/step - loss: 0.8545
Epoch 13/100
1/1 [==============================] - 0s 32ms/

2024-02-13 09:47:39.152687: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [231]:
epochs_to_look_at = agents_to_sample

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    if resnet:
        model = ResNet(board_size, path)
    else:
        model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY,
            epochs=100,
            dynamic=True
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}"):
        # Test if epoch folder exists
        if os.path.exists(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{epoch}"):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{epoch}"):
                os.remove(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{epoch}/{file}")

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/dynamic", exist_ok=True)
    os.makedirs(f"../concept_presences/dynamic/{model_type}", exist_ok=True)
    os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}", exist_ok=True)
    os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}", exist_ok=True)
    os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}", exist_ok=True)
    os.makedirs(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{epoch}", exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open(f"../concept_presences/dynamic/{model_type}/{board_name}/{session_name}/{CONCEPT_NAME}/{epoch}/{random_suffix}.json", "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)

Getting activation outputs: 100%|██████████| 1/1 [00:00<00:00,  2.31it/s]

Performing regression for layer 0
Epoch 1/100



2024-02-13 09:47:43.363965: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 09:47:43.395568: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 848ms/step - loss: 1.1248
Epoch 2/100
1/1 [==============================] - 0s 113ms/step - loss: 1.1236
Epoch 3/100
1/1 [==============================] - 0s 70ms/step - loss: 1.1224
Epoch 4/100
1/1 [==============================] - 0s 106ms/step - loss: 1.1212
Epoch 5/100
1/1 [==============================] - 0s 90ms/step - loss: 1.1200
Epoch 6/100
1/1 [==============================] - 0s 25ms/step - loss: 1.1188
Epoch 7/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1175
Epoch 8/100
1/1 [==============================] - 0s 35ms/step - loss: 1.1163
Epoch 9/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1151
Epoch 10/100
1/1 [==============================] - 0s 22ms/step - loss: 1.1139
Epoch 11/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1127
Epoch 12/100
1/1 [==============================] - 0s 22ms/step - loss: 1.1115
Epoch 13/100
1/1 [==============================] - 0s 26m

2024-02-13 09:47:47.011042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 09:47:47.394872: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 09:47:47.426754: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


...

2024-02-13 09:49:08.015961: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 09:49:08.373463: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 09:49:08.405681: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 836ms/step - loss: 1.2021
Epoch 2/100
1/1 [==============================] - 0s 68ms/step - loss: 1.1992
Epoch 3/100
1/1 [==============================] - 0s 86ms/step - loss: 1.1963
Epoch 4/100
1/1 [==============================] - 0s 95ms/step - loss: 1.1934
Epoch 5/100
1/1 [==============================] - 0s 56ms/step - loss: 1.1905
Epoch 6/100
1/1 [==============================] - 0s 46ms/step - loss: 1.1876
Epoch 7/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1847
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1818
Epoch 9/100
1/1 [==============================] - 0s 41ms/step - loss: 1.1790
Epoch 10/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1761
Epoch 11/100
1/1 [==============================] - 0s 62ms/step - loss: 1.1733
Epoch 12/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1704
Epoch 13/100
1/1 [==============================] - 0s 23ms/

2024-02-13 09:49:11.866572: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 09:49:12.224684: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-13 09:49:12.257469: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 890ms/step - loss: 1.2088
Epoch 2/100
1/1 [==============================] - 0s 97ms/step - loss: 1.2064
Epoch 3/100
1/1 [==============================] - 0s 87ms/step - loss: 1.2039
Epoch 4/100
1/1 [==============================] - 0s 41ms/step - loss: 1.2015
Epoch 5/100
1/1 [==============================] - 0s 61ms/step - loss: 1.1991
Epoch 6/100
1/1 [==============================] - 0s 88ms/step - loss: 1.1967
Epoch 7/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1943
Epoch 8/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1919
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 1.1895
Epoch 10/100
1/1 [==============================] - 0s 42ms/step - loss: 1.1871
Epoch 11/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1847
Epoch 12/100
1/1 [==============================] - 0s 40ms/step - loss: 1.1823
Epoch 13/100
1/1 [==============================] - 0s 27ms/

2024-02-13 09:49:15.811275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
